In [23]:
import tensorflow as tf
import numpy as np  
import pandas as pd
import pickle 

from tensorflow.keras.models import load_model

In [24]:
model = load_model("model.h5")

with open("standard_scalar.pkl", "rb") as file:
    scaler = pickle.load(file)

with open("leg_data_gender.pkl", "rb") as file:
    leg_data_gender = pickle.load(file)

with open("ohe_data_geo.pkl", "rb") as file:
    ohe_data_geo = pickle.load(file)


In [62]:
input_data = {
    'CreditScore': 600.0,
    'Age': 40,
    'Tenure': 5,
    'Balance': 50000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 60000,
    'Geography': 'France',
    'Gender': 'Male'
}

In [63]:
geo_data = ohe_data_geo.transform([[input_data['Geography']]]).toarray()
geo_data = pd.DataFrame(geo_data, columns=ohe_data_geo.get_feature_names_out(['Geography']))
geo_data

/Users/madhav.khurana@grofers.com/Documents/DS2025/DL_Project1/venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2691: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [64]:
data = pd.DataFrame(input_data, index=[0])
data['Gender'] = leg_data_gender.transform(data['Gender'])

In [65]:
data.drop("Geography", axis=1, inplace=True)


In [66]:
data

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Gender
0,600.0,40,5,50000,2,1,1,60000,1


In [67]:
data_final = pd.concat((data, geo_data), axis=1)
data_final

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Gender,Geography_France,Geography_Germany,Geography_Spain
0,600.0,40,5,50000,2,1,1,60000,1,1.0,0.0,0.0


In [69]:
# data_final.shape

data_final = data_final[['CreditScore',
 'Gender',
 'Age',
 'Tenure',
 'Balance',
 'NumOfProducts',
 'HasCrCard',
 'IsActiveMember',
 'EstimatedSalary',
 'Geography_France',
 'Geography_Germany',
 'Geography_Spain']]


In [70]:
data_final

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600.0,1,40,5,50000,2,1,1,60000,1.0,0.0,0.0


In [71]:
data_final.shape

(1, 12)

In [72]:
input_scaled = scaler.transform(data_final)
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.00134472, -0.41792108,
         0.80843615,  0.64920267,  0.97481699, -0.70296551,  1.00150113,
        -0.57946723, -0.57638802]])

In [73]:
prediction = model.predict(input_scaled)
prediction

1/1 [==============================] - 0s 21ms/step


array([[0.03791164]], dtype=float32)

In [76]:
prediction_probability = prediction[0,0]
prediction_probability

0.03791164

In [77]:
if prediction_probability > 0.5:
    print("Likely to Churn")
else: print("Unlikely to Churn")

Unlikely to Churn
